In [ ]:
import sys
sys.path.append("../")

import sympy as sp
import pathlib as pl
from SymEigen import *
from sympy import symbols
from project_dir import backend_source_dir
from affine_body_core import compute_J_point, compute_J_vec

Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")
Gen.DisableLatexComment()

# Articulation

All stuff we need in optimization: 

$$
K = \frac{1}{2}(\delta\theta_i - \tilde{\delta\theta}_i) M_{ij} (\delta\theta_j - \tilde{\delta\theta}_j),
$$

$$
G^{\theta}_i = \frac{\partial K}{\partial \delta\theta_i} = M_{ij} (\delta\theta_j - \tilde{\delta\theta}_j),
$$

$$
H^{\theta}_{ij} = \frac{\partial^2 K}{\partial \delta\theta_i \partial \delta\theta_j} = M_{ij},
$$

$$
\begin{aligned}
G^{F}_i 
&= \frac{\partial K}{\partial \mathbf{F}_i} \\
&= \frac{\partial K}{\partial \delta\theta_i} \frac{\partial \delta\theta_i}{\partial \mathbf{F}_i} \quad (\text{no-ein}) \\
&= G^{\theta}_i \frac{\partial \delta\theta_i}{\partial \mathbf{F}_i} \quad (\text{no-ein}) \\
\end{aligned}
$$

$$
\begin{aligned}
H^{F}_{ij} 
&= \frac{\partial^2 K}{\partial \mathbf{F}_i \partial \mathbf{F}_j}\\
&= \frac{\partial \theta_i}{\partial \mathbf{F}_i} \frac{\partial^2 K}{\partial \theta_i \partial \theta_j} \frac{\partial \theta_j}{\partial \mathbf{F}_j} + \delta_{ij} \frac{\partial K}{\partial \theta_i} \frac{\partial^2 \theta_i}{\partial \mathbf{F}_i \partial \mathbf{F}_i} \quad (\text{no-ein}) \\
&= \frac{\partial \theta_i}{\partial \mathbf{F}_i} M_{ij} \frac{\partial \theta_j}{\partial \mathbf{F}_j} + \delta_{ij} G^{\theta}_i \frac{\partial^2 \theta_i}{\partial \mathbf{F}_i \partial \mathbf{F}_i} \quad (\text{no-ein}) \\
\end{aligned}
$$

For symbol calculation:

$$
\frac{\partial \delta\theta}{\partial \mathbf{F}}
$$

$$
\frac{\partial^2 \delta\theta}{\partial \mathbf{F} \partial \mathbf{F}}
$$

# Mapping to Affine Body Space

$$
G^{q} = J^T G^{F}
$$

$$
H^{q} = J^T H^{F} J
$$

In [ ]:
# two affine body i and j
qk = Eigen.Vector("qk",12)
qk_t = Eigen.Vector("qk_t",12)
ql = Eigen.Vector("ql",12)
ql_t = Eigen.Vector("ql_t",12)

# Revolute Joint Constraint

![Revolute Joint Constraint](../../docs/specification/constitutions/media/external_articulation_revolute_constraint_fig1.drawio.svg)

We define the joint frame $\mathbf{F}$

$$
\mathbf{F} =
\begin{bmatrix}
\hat{\mathbf{b}}_i \\
\hat{\mathbf{n}}_i \\
\hat{\mathbf{b}}_j \\
\hat{\mathbf{n}}_j
\end{bmatrix}
$$

In [ ]:
def compute_cos_theta(bi, ni, bj, nj):
    return (bi.dot(bj) + ni.dot(nj)) / 2

def compute_sin_theta(bi, ni, bj, nj):
    return (ni.dot(bj) - bi.dot(nj)) / 2

def compute_revolute_delta_theta(F,F_t):
    bi = F[0:3, 0]
    ni = F[3:6, 0]
    bj = F[6:9, 0]
    nj = F[9:12, 0]
    bi_t = F_t[0:3, 0]
    ni_t = F_t[3:6, 0]
    bj_t = F_t[6:9, 0]
    nj_t = F_t[9:12, 0]
    cos_theta = compute_cos_theta(bi, ni, bj, nj)
    sin_theta = compute_sin_theta(bi, ni, bj, nj)
    cos_theta_t = compute_cos_theta(bi_t, ni_t, bj_t, nj_t)
    sin_theta_t = compute_sin_theta(bi_t, ni_t, bj_t, nj_t)
    tan_delta_theta = (sin_theta * cos_theta_t - cos_theta * sin_theta_t) / (cos_theta * cos_theta_t + sin_theta * sin_theta_t)
    delta_theta = sp.atan(tan_delta_theta)
    return delta_theta

# rest-shape space vectors
# [b_bar, n_bar]
basis_k = Eigen.Vector("basis_k", 6)
basis_l = Eigen.Vector("basis_l", 6)
basis_m = Eigen.Vector("basis_m", 6)
basis_n = Eigen.Vector("basis_n", 6)

From affine body $i$ and $j$ to $\mathbf{F}$:

$$
\begin{aligned}
J_r 
&= \frac{\partial \mathbf{F}}{\partial \mathbf{q}} \\
&=
\begin{bmatrix}
\mathring{\mathbf{J}}(\bar{\mathbf{b}}_k) &  \\
\mathring{\mathbf{J}}(\bar{\mathbf{n}}_k) &  \\
& \mathring{\mathbf{J}}(\bar{\mathbf{b}}_l)  \\
& \mathring{\mathbf{J}}(\bar{\mathbf{n}}_l)  \\
\end{bmatrix}_{12 \times 24} 
\end{aligned}
$$

In [ ]:
# build J_F
def compute_J(basis_k, basis_l):
    Jr = sp.Matrix.zeros(12, 24)
    bk_bar = basis_k[0:3, 0]
    nk_bar = basis_k[3:6, 0]
    bl_bar = basis_l[0:3, 0]
    nl_bar = basis_l[3:6, 0]
    Jr[0:3, 0:12] = compute_J_vec(bk_bar)
    Jr[3:6, 0:12] = compute_J_vec(nk_bar)
    Jr[6:9, 12:24] = compute_J_vec(bl_bar)
    Jr[9:12, 12:24] = compute_J_vec(nl_bar)
    return Jr

Jr_i = compute_J(basis_k, basis_l)
Jr_j = compute_J(basis_m, basis_n)

# compute F from ABD qk, ql
Fr_q = Jr_i @ sp.Matrix.vstack(qk, ql)
Fr_q_t = Jr_i @ sp.Matrix.vstack(qk_t, ql_t)

delta_theta = compute_revolute_delta_theta(Fr_q, Fr_q_t)
dDeltaTheta_dQ = VecDiff(delta_theta, sp.Matrix.vstack(qk, ql))

Gr = Eigen.Vector("G_F", 12)
Hr = Eigen.Matrix("H_F", 12, 12)

JrT_G = Jr_i.T @ Gr
JrT_H_Jr = Jr_i.T @ Hr @ Jr_j

Cl_DeltaTheta = Gen.Closure(basis_k, qk, qk_t, basis_l, ql, ql_t)

Cl_Fr = Gen.Closure(basis_k, qk, # Affine Body k
                    basis_l, ql) # Affine Body l

Cl_Gr = Gen.Closure(Gr, 
                    basis_k, # Affine Body k
                    basis_l) # Affine Body l

Cl_Hr = Gen.Closure(Hr, 
                    basis_k, # Affine Body k
                    basis_l, # Affine Body l
                    basis_m, # Affine Body m
                    basis_n) # Affine Body n

content = ""
content += f""" 
// External Revolute Joint Constraint
// Description: Symbolic expressions for revolute joint constraint between two affine bodies.
// Affine Body <-> Revolute Joint Frame Mapping

// F = J_r * [qi; qj]
{Cl_Fr("F", Fr_q)}
// DeltaTheta
{Cl_DeltaTheta("DeltaTheta", delta_theta)}
// d(DeltaTheta)/dQ
{Cl_DeltaTheta("dDeltaTheta_dQ", dDeltaTheta_dQ)}
// JT_G = J_r^T * G_F
{Cl_Gr("JT_G", JrT_G)}
// JT_H_J = J_r^T * H_F * J_r
{Cl_Hr("JT_H_J", JrT_H_Jr)}
"""

In [ ]:
F = Eigen.Vector("F", 12)
F_t = Eigen.Vector("F_t", 12)
Cl = Gen.Closure(F, F_t)

$$
\cos\theta = \frac{
    \hat{\mathbf{b}}_i \cdot \hat{\mathbf{b}}_j + \hat{\mathbf{n}}_i \cdot \hat{\mathbf{n}}_j
    }{2},
$$

$$
\sin\theta = \frac{
    \hat{\mathbf{n}}_i \cdot \hat{\mathbf{b}}_j - \hat{\mathbf{b}}_i \cdot \hat{\mathbf{n}}_j
    }{2}.
$$

$$
\delta\theta = \arctan \left(\tan \left(\theta - \theta^t \right)\right) = \arctan \frac{\sin\theta \cos\theta^t - \cos\theta \sin\theta^t}{\cos\theta \cos\theta^t + \sin\theta \sin\theta^t},
$$

In [ ]:
delta_theta = compute_revolute_delta_theta(F, F_t)
d_delta_theta_dF = VecDiff(delta_theta, F)
dd_delta_theta_ddF = VecDiff(d_delta_theta_dF, F)

In [ ]:
content += f"""
// Theta and Derivatives for External Revolute Joint Constraint

{Cl("DeltaTheta", delta_theta)}
{Cl("dDeltaTheta_dF",d_delta_theta_dF)}
{Cl("ddDeltaTheta_ddF",dd_delta_theta_ddF)}
"""

path = backend_source_dir("cuda") / "affine_body/constraints/sym/external_articulation_revolute_joint_constraint.inl"
print("Written to:", path)
f = open(path, "w")
f.write(content)
f.close()

# Prismatic Joint Constraint

![Prismatic Joint Constraint](../../docs/specification/constitutions/media/external_articulation_prismatic_constraint_fig1.drawio.svg)

$$
    \theta = \frac{
        (\mathbf{c}_j - \mathbf{c}_i) \cdot \hat{\mathbf{t}}_i -
        (\mathbf{c}_i - \mathbf{c}_j) \cdot \hat{\mathbf{t}}_j
    }{2},
$$

We define the joint frame $\mathbf{F}$

$$
\mathbf{F} =
\begin{bmatrix}
\mathbf{c}_i \\
\hat{\mathbf{t}}_i \\
\mathbf{c}_j \\
\hat{\mathbf{t}}_j
\end{bmatrix}
$$

In [ ]:
def compute_prismatic_theta(F):
    ci = F[0:3, 0]
    ti = F[3:6, 0]
    cj = F[6:9, 0]
    tj = F[9:12, 0]
    dij = cj - ci
    theta = (dij.dot(ti) + dij.dot(tj))/2
    return theta

def compute_prismatic_delta_theta(F, F_t):
    theta = compute_prismatic_theta(F)
    theta_t = compute_prismatic_theta(F_t)
    delta_theta = theta - theta_t
    return delta_theta

# basis = [c_bar, t_bar]
basis_l = Eigen.Vector("basis_l", 6)
basis_k = Eigen.Vector("basis_k", 6)
basis_m = Eigen.Vector("basis_m", 6)
basis_n = Eigen.Vector("basis_n", 6)

From affine body $i$ and $j$ to $\mathbf{F}$:

$$
\begin{aligned}
J_p 
&= \frac{\partial \mathbf{F}}{\partial \mathbf{q}} \\
&=
\begin{bmatrix}
\mathbf{J}(\bar{\mathbf{c}}_k) & \\
\mathring{\mathbf{J}}(\bar{\mathbf{t}}_k) &  \\
& \mathbf{J}(\bar{\mathbf{c}}_l) \\
& \mathring{\mathbf{J}}(\bar{\mathbf{t}}_l) \\
\end{bmatrix}_{12 \times 24} 
\end{aligned}
$$

In [ ]:
def compute_J(basis_i, basis_j):
    Jp = sp.Matrix.zeros(12, 24)
    ci_bar = basis_i[0:3, 0]
    ti_bar = basis_i[3:6, 0]
    cj_bar = basis_j[0:3, 0]
    tj_bar = basis_j[3:6, 0]
    Jp[0:3, 0:12] = compute_J_point(ci_bar)
    Jp[3:6, 0:12] = compute_J_vec(ti_bar)
    Jp[6:9, 12:24] = compute_J_point(cj_bar)
    Jp[9:12, 12:24] = compute_J_vec(tj_bar)
    return Jp

Jp_i = compute_J(basis_k, basis_l)
Jp_j = compute_J(basis_m, basis_n)

# compute F from ABD qk, ql
Fp_q = Jp_i @ sp.Matrix.vstack(qk, ql)
Fp_q_t = Jp_i @ sp.Matrix.vstack(qk_t, ql_t)

Gp = Eigen.Vector("Gp", 12)
Hp = Eigen.Matrix("Hp", 12, 12)

JpT_G = Jp_i.T @ Gp
JpT_H_Jp = Jp_i.T @ Hp @ Jp_j

delta_theta = compute_prismatic_delta_theta(Fp_q, Fp_q_t)
dDeltaTheta_dQ = VecDiff(delta_theta, sp.Matrix.vstack(qk, ql))

display( delta_theta )
display( dDeltaTheta_dQ )

Cl_DeltaTheta_p = Gen.Closure(basis_k, qk, qk_t, basis_l, ql, ql_t)

Cl_Fp = Gen.Closure(basis_k, qk, # Affine Body k
                    basis_l, ql) # Affine Body l

Cl_Gp = Gen.Closure(Gp, 
                    basis_k, # Affine Body k
                    basis_l) # Affine Body l

Cl_Hp = Gen.Closure(Hp, 
                    basis_k, # Affine Body k 
                    basis_l, # Affine Body l
                    basis_m, # Affine Body m
                    basis_n) # Affine Body n

content = ""
content += f""" 
// External Prismatic Joint Constraint
// Description: Symbolic expressions for prismatic joint constraint between two affine bodies.
// Affine Body <-> Prismatic Joint Frame Mapping

// F = J_p * [qi; qj]
{Cl_Fp("F", Fp_q)}
// DeltaTheta
{Cl_DeltaTheta_p("DeltaTheta", delta_theta)}
// d(DeltaTheta)/dQ
{Cl_DeltaTheta_p("dDeltaTheta_dQ", dDeltaTheta_dQ)}
// JT_G = J_p^T * G_p
{Cl_Gp("JT_G", JpT_G)}
// JT_H_J = J_p^T * H_p * J_p
{Cl_Hp("JT_H_J", JpT_H_Jp)}
"""

In [ ]:
F = Eigen.Vector("F", 12)
F_t = Eigen.Vector("F_t", 12)

Cl = Gen.Closure(F, F_t)

delta_theta = compute_prismatic_delta_theta(F, F_t)
d_delta_theta_dF = VecDiff(delta_theta, F)
dd_delta_theta_ddF = VecDiff(d_delta_theta_dF, F)

content += f"""// Affine Body Prismatic Joint Constraint
// Description: Symbolic expressions for prismatic joint constraint between two affine bodies.

{Cl("DeltaTheta", delta_theta)}
{Cl("dDeltaTheta_dF", d_delta_theta_dF)}
{Cl("ddDeltaTheta_ddF", dd_delta_theta_ddF)}
"""


path = backend_source_dir("cuda") / "affine_body/constraints/sym/external_articulation_prismatic_joint_constraint.inl"
print("Written to:", path)
f = open(path, "w")
f.write(content)
f.close()

$$
\| \delta \theta(q_{u},q^t_{gs}) - \tilde{\delta \theta}_{gs} \|_M^2
$$

$$
q^t_{gs} \leftarrow (x, quat)_{gs}
$$